---

# 🔧 TypeAdapter Validation

### 🎯 Intent

Validate **standalone Python types** (lists, dicts, tuples, etc.) without creating a `BaseModel`.

---

### 🧩 Core Components

1. **📝 Create TypeAdapter**

   * Define a `TypeAdapter` for any Python type.
   * Example: list, dict, tuple, union types.

2. **⚡ Methods**

   * `.validate_python(obj)` → validate Python objects.
   * `.validate_json(str)` → validate JSON input.

3. **📦 Coercion**

   * Converts compatible values automatically.
   * Example: `["1", 2]` → `[1, 2]`.

4. **🛑 Error Handling**

   * Raises `ValidationError` if input doesn’t match type.

5. **🌍 Use Cases**

   * Quick validation for configs, API payloads, ETL pipelines, etc.
   * Avoids writing a full model when not needed.

---
